# SIB Renewal Schemes


In [1]:
#!pip install -qqqqq pandas jinja2 numpy google-auth openpyxl gspread

Library Imports


In [ ]:
import pandas as pd
import json,numpy as np
import jinja2   
from jinja2 import Template 
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from decimal import Decimal, getcontext, ROUND_HALF_UP
from typing import Optional
import gspread
from google.oauth2.service_account import Credentials

SIB renewals calculator


In [3]:
#sib caluclator
def sib_renewals_calc(
    tenure_months: Optional[Decimal],
    cx_ltv: Optional[Decimal],
    cl1_principal_share_pct :Optional[Decimal],
    cl2_principal_share_pct:Optional[Decimal],
    pf_min_pct: Optional[Decimal],
    pf_max_pct: Optional[Decimal],
    s1_monthly_int_rate_pct: Optional[Decimal]
):
    #Internal Helper for safe Decimal conversion
    def safe_d(val) -> Decimal:
        return Decimal(str(val)) if val is not None else Decimal("0")

    #Normalize Inputs
    cx_ltv = safe_d(cx_ltv)
    cl1_principal_share_pct = safe_d(cl1_principal_share_pct) / Decimal("100")
    cl2_principal_share_pct = safe_d(cl2_principal_share_pct) / Decimal("100")
    pf_min = safe_d(pf_min_pct)
    pf_max = safe_d(pf_max_pct)
    s1_monthly_rate = safe_d(s1_monthly_int_rate_pct)
    
    #Core Math
    secure_overall_ltv = cx_ltv
    secure1_ltv = cl1_principal_share_pct * cx_ltv
    secure2_ltv = secure_overall_ltv - secure1_ltv

    s1_annual_rate = (s1_monthly_rate * Decimal("12")) / Decimal("100")
    s2_annual_rate = Decimal("0.20")
    s3_annual_rate = Decimal("0.24") 
    cl1_addon_rate = Decimal("0.0995")

    #Formula logic
    ir_0_30 = ((s1_annual_rate - (cl1_addon_rate * cl1_principal_share_pct)) / cl2_principal_share_pct) * 100
    ir_31_60 = ((s2_annual_rate - (cl1_addon_rate * cl1_principal_share_pct)) / cl2_principal_share_pct) * 100
    ir_61_360 = ((s3_annual_rate - (cl1_addon_rate * cl1_principal_share_pct)) / cl2_principal_share_pct) * 100

    min_total = pf_min / cl2_principal_share_pct
    max_total = pf_max / cl2_principal_share_pct

    #Formatting Helper
    def q2(x: Decimal) -> str:
        return f"{x.quantize(Decimal('0.01'), rounding=ROUND_HALF_UP):.2f}"

    return {
        "cx_ltv": q2(cx_ltv),
        "s1_annual_rate":q2(s1_annual_rate*100),
        "s2_annual_rate":q2(s2_annual_rate*100),
        "s3_annual_rate":q2(s3_annual_rate),
        "tenure_months": str(tenure_months) if tenure_months else "0",
        "secure_overall_ltv": q2(secure_overall_ltv),
        "secure1_ltv": q2(secure1_ltv),
        "secure2_ltv": q2(secure2_ltv),
        "int_rate_day_0_30": q2(ir_0_30),
        "int_rate_day_31_60": q2(ir_31_60),
        "int_rate_day_61_360": q2(ir_61_360),
        "cl1_principal_share": q2(cl1_principal_share_pct * 100),
        "cl2_principal_share": q2(cl2_principal_share_pct * 100),
        "min_percent_total": q2(min_total),
        "max_percent_total": q2(max_total),
        "s1_monthly_int_rate_pct": q2(s1_monthly_rate),
        "s3_annual_rate": q2(s3_annual_rate * 100),
        "pf_min_pct": q2(pf_min),
        "pf_max_pct": q2(pf_max)
    }

In [4]:
# sib_renewals_calc(                    #testing the calculator function
#     tenure_months=36,
#     cx_ltv=84,
#     cl1_principal_share_pct=80,
#     cl2_principal_share_pct=20,
#     pf_max_pct=0.10,
#     pf_min_pct=1.00,
#     s1_monthly_int_rate_pct=1.15)

LOADING THE SIB RENEWALS TEMPLATE


In [5]:
def load_templates(
    service_account_json: str,
    spreadsheet_id: str,
    worksheet_name: str,
):
    """
    Reads a Google Sheet tab using a service account and
    returns list[dict] (same output as your CSV version)
    """

    scopes = [
        "https://www.googleapis.com/auth/spreadsheets.readonly"
    ]

    creds = Credentials.from_service_account_file(
        service_account_json,
        scopes=scopes
    )

    client = gspread.authorize(creds)

    worksheet = client.open_by_key(spreadsheet_id).worksheet(worksheet_name)

    records = worksheet.get_all_records()
    df = pd.DataFrame(records)

    df = df.dropna(how="all", axis=0)

    return df.to_dict(orient="records")


LOADING THE SAMPLE CSV

In [6]:
def load_sample_input(file_path: str):
    """
    Loads the sample.csv exactly as it is without changing header casing.
    """
    # 1. Read the CSV directly
    column_dtypes = {
        "template_type": str,
        'cx_ltv':int,
        "cl1_principal_share_pct": float,
        "cl2_principal_share_pct": float,
        "s1_monthly_int_rate_pct": float,        
        'city_id': str,
        'overall_secure_ltv':int,
        'pf_min_pct':float,
        'pf_max_pct':float,
        'ts_min':int,
        'ts_max':int}
        
    df = pd.read_csv(file_path,dtype=column_dtypes)    
    df.columns = [str(c).replace("\ufeff", "").strip() for c in df.columns]
    df = df.dropna(how="all", axis=0)
    df=df.dropna(how="all", axis=1)
    input_rows = df.to_dict(orient="records")
    return df.to_dict(orient="records")

LTV CODE


In [7]:
def get_ltv_code(ltv: int) -> str:
    mapping = {
        70: "s0", 71: "s1", 72: "s2", 73: "s3", 74: "s4", 
        75: "s5", 76: "s6", 77: "s7", 78: "s8", 79: "s9",
        80: "e0", 81: "e1", 82: "e2", 83: "e3", 84: "e4", 
        85: "e5", 86: "e6", 87: "e7", 88: "e8", 89: "e9",
        90: "n0"
    }
    
    if ltv in mapping:
        return mapping[ltv]
    
    raise ValueError(f"No LTV code defined for ltv={ltv}")

TS slab in refname


In [8]:
def get_slab_label(ts_min: int, ts_max: int) -> str:
    if ts_max < 300000:
        return "<3L"

    if 300000 <= ts_min and ts_max <= 599999:
        return "3-6L"
    if 600000 <= ts_min and ts_max <= 1199999:
        return "6-12L"
    if 1200000 <= ts_min and ts_max <= 2499999:
        return "12-25L"
    if ts_max >= 2500000:
        return ">25L"

    raise ValueError(f"Unknown slab")

end tags


In [9]:
#adding any tag at the end of refname
def tagged(tag):
    if pd.isna(tag) or tag is None:
        return ""
    return str(tag).upper()

HIghlight


In [10]:
HIGHLIGHT_COLS = ['refName','SchemeMin','SchemeMax','tenure','customerLtv','OverallInterestCalculation','OverallPF','bs1-legalName','bs1-ltv','bs1-charge-2','bs1-addon-1','bs2-addon-1','cl2-legalName','cl2-charge-2','cl2-addon-1']

In [11]:
def highlight_columns(excel_path:str,
                      cols_to_hightlight:list[str],
                      color:str= "FFFF00"):
    wb=load_workbook(excel_path)
    ws=wb.active

    fill=PatternFill(start_color=color,end_color=color,fill_type="solid")

    header_to_col = {
        cell.value: idx + 1
        for idx, cell in enumerate(ws[1])
        if cell.value
    }

    for col_name in cols_to_hightlight:
        if col_name not in header_to_col:
            print(f"⚠️ Column not found, skipping: {col_name}")
            continue

        col_idx = header_to_col[col_name]
        for row in range(1, ws.max_row + 1):
            ws.cell(row=1, column=col_idx).fill = fill

    wb.save(excel_path)

Scheme validator


In [12]:
def sib_scheme_validator(
    tenure_months: Optional[Decimal],
    cx_ltv: Optional[Decimal],
    cl1_principal_share_pct: Optional[Decimal],
    cl2_principal_share_pct: Optional[Decimal],
    pf_min_pct: Optional[Decimal],
    pf_max_pct: Optional[Decimal],
    s1_monthly_int_rate_pct: Optional[Decimal],
    int_rate_day_0_30: Optional[Decimal],
    int_rate_day_31_60: Optional[Decimal],
    int_rate_day_61_360: Optional[Decimal],
):
    errors = []
    TWOPLACES = Decimal("0.01")

    # -------------------------
    # Safe Decimal conversion
    # -------------------------
    def safe_d(val, default=None):
        if val is None:
            return default
        s = str(val).strip().lower()
        if s in ("", "nan"):
            return default
        return Decimal(s)

    # Normalize inputs
    
    cx_ltv = safe_d(cx_ltv)
    tenure_months = safe_d(tenure_months)

    cl1_share = safe_d(cl1_principal_share_pct)
    cl2_share = safe_d(cl2_principal_share_pct)

    pf_min = safe_d(pf_min_pct, Decimal("0"))
    pf_max = safe_d(pf_max_pct, Decimal("0"))

    s1_monthly_rate = safe_d(s1_monthly_int_rate_pct)

    
    # Basic presence checks
    
    if cx_ltv is None or cx_ltv <= 0:
        errors.append("CX LTV must be > 0")

    if tenure_months is None or tenure_months <= 0:
        errors.append("Tenure must be > 0")

    if cl1_share is None or cl2_share is None:
        errors.append("Principal shares missing")
        return {"status": "INVALID", "errors": errors}

    # convert % → fraction
    cl1_share = cl1_share / Decimal("100")
    cl2_share = cl2_share / Decimal("100")

    if cl1_share <= 0 or cl2_share <= 0:
        errors.append("Principal shares must be > 0")

    if (cl1_share + cl2_share).quantize(TWOPLACES) != Decimal("1.00"):
        errors.append("Principal shares must sum to 100%")

    if s1_monthly_rate is None or s1_monthly_rate <= 0:
        errors.append("Monthly interest rate must be > 0")

    # Constants (business rules)

    cl1_addon_rate = Decimal("0.0995")   # 9.95%
    s2_annual_rate = Decimal("0.20")     # 20%
    s3_annual_rate = Decimal("0.24")     # 24%

    s1_annual_rate = (s1_monthly_rate * Decimal("12")) / Decimal("100")

    # Reverse math: derive slab IRs

    try:
        ir_0_30 = (
            (s1_annual_rate - (cl1_addon_rate * cl1_share)) / cl2_share
        ) * 100

        ir_31_60 = (
            (s2_annual_rate - (cl1_addon_rate * cl1_share)) / cl2_share
        ) * 100

        ir_61_360 = (
            (s3_annual_rate - (cl1_addon_rate * cl1_share)) / cl2_share
        ) * 100
    except Exception:
        errors.append("Division error while deriving interest slabs")

    # Reverse validation helper
   
    def check_blended(expected_annual, slab_rate, label):
        reconstructed = (
            (cl1_addon_rate * cl1_share) +
            ((slab_rate / Decimal("100")) * cl2_share)
        ).quantize(TWOPLACES)

        expected = expected_annual.quantize(TWOPLACES)

        if reconstructed != expected:
            errors.append(
                f"{label} reverse check failed "
                f"(expected {expected}, got {reconstructed})"
            )
    # Reverse checks: slabs
    
    check_blended(s1_annual_rate, ir_0_30, "IR 0–30")
    check_blended(s2_annual_rate, ir_31_60, "IR 31–60")
    check_blended(s3_annual_rate, ir_61_360, "IR 61–360")

    # Final verdict

    if errors:
        return {
            "status": "INVALID",
            "errors": errors
        }
    return {
        "status": "VALID"}

Scheme Generator


In [13]:
def generate_schemes(input_path, output_path):
    print("✅ Initializing Scheme Generation")

    # 1. Loading Data

    print(f"✅ Loading input file: {input_path}")
    input_rows = load_sample_input(input_path)

    print(f"✅ Loading template file fromt the google sheet")
    
    SERVICE_ACCOUNT_JSON = r"C:\Users\Ingit.Paul.in\Downloads\schemes-486410-773d0189a4ce.json"   #upload the service account json file path here
    SPREADSHEET_ID = "18QsZeCu0vEET6p-wnxhAljWG8JpQOUTqwVhmJSesN00" #dont touch
    WORKSHEET_NAME = "SIB_RENEWALS"                                 #dont touch

    template_rows = load_templates(SERVICE_ACCOUNT_JSON,SPREADSHEET_ID,WORKSHEET_NAME)
    
    final_schemes = []
    total_rows = len(input_rows)
    print(f"✅ Data loaded. Found {total_rows} rows to process.")
    print("-" * 50)

    for i, row in enumerate(input_rows, 1):
        # Progress indicator
        print(f"🔄 Processing Row [{i}/{total_rows}] | Type: {row.get('template_type')}...", end="\r")

        #Helper to safely get float values
        def get_decimal_or_none(row, key):
            val = row.get(key)
            if val is None or str(val).strip().lower() in ("nan", ""):
                return None   
            return val                   
        

        #Logic Mapping (LTV & Slab)
        current_ltv = int(row.get("cx_ltv"))
        ltv_code = get_ltv_code(current_ltv)          
        overall_secure_ltv = row.get("overall_secure_ltv")
        s1_monthly_int_rate_pct= row.get("s1_monthly_int_rate_pct")        
        ts_min = int(row.get("ts_min"))
        ts_max = int(row.get("ts_max"))
        slab_label = get_slab_label(ts_min, ts_max)      

        #city
        city_id = row.get("city_id")    
        
        #Run Calculation (sib_renewals_calc)

        calc_results = sib_renewals_calc(
            tenure_months=get_decimal_or_none(row, "tenure_months"),
            cx_ltv=get_decimal_or_none(row, "cx_ltv"),
            cl1_principal_share_pct=get_decimal_or_none(row, "cl1_principal_share_pct"),
            cl2_principal_share_pct=get_decimal_or_none(row, "cl2_principal_share_pct"),
            pf_min_pct=get_decimal_or_none(row, "pf_min_pct"),
            pf_max_pct=get_decimal_or_none(row, "pf_max_pct"),
            s1_monthly_int_rate_pct=get_decimal_or_none(row, "s1_monthly_int_rate_pct"))

        # validation   
        validation_result = sib_scheme_validator(
            tenure_months=get_decimal_or_none(row, "tenure_months"),
            cx_ltv=get_decimal_or_none(row, "cx_ltv"),
            cl1_principal_share_pct=get_decimal_or_none(row, "cl1_principal_share_pct"),
            cl2_principal_share_pct=get_decimal_or_none(row, "cl2_principal_share_pct"),
            pf_min_pct=get_decimal_or_none(row, "pf_min_pct"),
            pf_max_pct=get_decimal_or_none(row, "pf_max_pct"),
            s1_monthly_int_rate_pct=get_decimal_or_none(row, "s1_monthly_int_rate_pct"),
            int_rate_day_0_30=calc_results["int_rate_day_0_30"],
            int_rate_day_31_60=calc_results["int_rate_day_31_60"],
            int_rate_day_61_360=calc_results["int_rate_day_61_360"])
        
        if validation_result["status"] == "INVALID":
            error_msg = ", ".join(validation_result["errors"])
            print(f"\n❌ Skipping Row {i} | Error: {error_msg}")
            continue
              
        # 4. Build Context (Adding the new labels here)
        context = {
            **row,
            **calc_results,
            "ltv_code": ltv_code,
            "slab_label": slab_label,
            "city_id": city_id
        }
        print(f"Values to be inserted for Row {i}: {context}")
        
        #cleaning the NaN values
        for k, v in context.items():
            if pd.isna(v):
                context[k] = ""
            elif isinstance(v, float):
                context[k] = f"{v:.2f}"
            else:
                context[k] = v

        # Matching & Rendering templates
        t_type = row.get("template_type")
        template = next((t for t in template_rows if t.get("template_type") == t_type), None)

        if template:
            rendered_scheme = {}
            for key, value in template.items():
                if isinstance(value, str) and "{{" in value:
                    rendered_scheme[key] = Template(value).render(context)
                else:
                    rendered_scheme[key] = value
            
            final_schemes.append(rendered_scheme)
        else:
            print(f"\n⚠️ Warning: No template match for type '{t_type}' at row {i}")

    print(f"\n\n✨ All rows processed!")
    print("-" * 50)

    # 6. Output to New File
    if final_schemes:
        print(f"✅Saving results to: {output_path}")
        df_output = pd.DataFrame(final_schemes)
        
        if "template_type" in df_output.columns:
            df_output = df_output.drop(columns=["template_type"])
            
        df_output.to_excel(output_path, index=False)
        print(f"🎊 Success! {len(final_schemes)} schemes generated successfully.")
        
        #Highlighting important columns      

        highlight_columns(excel_path=output_path,cols_to_hightlight=HIGHLIGHT_COLS)

        print("✅Check the Highlighted columns in the output Excel file.")
    else:
        print("❌Error: No schemes were generated. Please check your template_type values.")

if __name__ == "__main__":
    generate_schemes(
        input_path=r"c:\jinja_schemes\sib renewals\sample_sib_renewals.csv",          ## UPLOAD THE CSV FILE FROM HERE
        output_path=r"C:\jinja_schemes\output17.xlsx"                                 ## THIS IS WHERE YOU WANT TO SAVE THE OUTPUT FILE
    )

✅ Initializing Scheme Generation
✅ Loading input file: c:\jinja_schemes\sib renewals\sample_sib_renewals.csv
✅ Loading template file fromt the google sheet
✅ Data loaded. Found 3 rows to process.
--------------------------------------------------
Values to be inserted for Row 1: {'template_type': 'SIB_IG', 'cx_ltv': '76.00', 'pf_min_pct': '0.10', 'pf_max_pct': '1.00', 'ts_min': 25000, 'ts_max': 299999, 'tenure_months': '12', 'cl1_principal_share_pct': 77.88, 'cl2_principal_share_pct': 22.12, 's1_monthly_int_rate_pct': '1.15', 's1_annual_rate': '13.80', 's2_annual_rate': '20.00', 's3_annual_rate': '24.00', 'secure_overall_ltv': '76.00', 'secure1_ltv': '59.19', 'secure2_ltv': '16.81', 'int_rate_day_0_30': '27.36', 'int_rate_day_31_60': '55.38', 'int_rate_day_61_360': '73.47', 'cl1_principal_share': '77.88', 'cl2_principal_share': '22.12', 'min_percent_total': '0.45', 'max_percent_total': '4.52', 'ltv_code': 's6', 'slab_label': '<3L', 'city_id': None}
Values to be inserted for Row 2: {'te

# End of the loop
